In [11]:
import numpy as np
import torch
import torch.nn as nn
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler
from tqdm import tqdm
from PIL import Image

In [12]:
class VGG16(nn.Module):
  def __init__(self, numClasses = 10):
    super(VGG16, self).__init__()
    self.features = nn.Sequential(
        # block 1
        nn.Conv2d(3, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.Conv2d(64, 64, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(64),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 2
        nn.Conv2d(64, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.Conv2d(128, 128, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(128),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 3
        nn.Conv2d(128, 256, kernel_size = 3, stride = 1, padding  = 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.Conv2d(256, 256, kernel_size=3, padding=1),
        nn.BatchNorm2d(256),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 4
        nn.Conv2d(256, 512, kernel_size = 3, stride = 1, padding  = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2),
        # block 5
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding  = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.Conv2d(512, 512, kernel_size = 3, stride = 1, padding = 1),
        nn.BatchNorm2d(512),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size = 2, stride = 2)
    )
    self.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(7 * 7 * 512, 4096),
        nn.ReLU(),
        nn.Dropout(0.5),
        nn.Linear(4096, 4096),
        nn.ReLU(),
        nn.Linear(4096, numClasses)
    )
  def forward(self, out):
    out = self.features(out)
    out = out.reshape(out.size(0), -1)
    out = self.classifier(out)
    return out


In [13]:
numClasses = 10
numEpochs = 20
learningRate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = VGG16(numClasses).to(device)

In [14]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2033, 0.1994, 0.2010])
])

In [87]:
classNames = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
model = VGG16(numClasses)
model.load_state_dict(torch.load('model_epoch_20.pth', map_location=torch.device('cpu')))
image = Image.open("test3.jpg")
image = transform(image)
image = image.unsqueeze(0) # Thêm một chiều batch để phù hợp với đầu vào của mô hình
outputs = model(image)
_, predicted = torch.max(outputs, 1)  # Lấy chỉ số của lớp có xác suất cao nhất
print(classNames[predicted.item()])

dog
